<a href="https://colab.research.google.com/github/Domsheen/learning-coding-projects/blob/main/unit_2__multiagent_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [20]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido duckduckgo-search -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.8 MB/s eta 0:00:00


In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [23]:
import os
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, VisitWebpageTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [24]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [25]:
# modified the intake process for adding the Serper API key to make it hidden

import os
from getpass import getpass

try:
    # ↑ if we’re in Colab and you’ve added them via Tools → User secrets…
    from google.colab import userdata

    os.environ["SERPAPI_API_KEY"] = userdata.get("SERPAPI_API_KEY")
    print("🔒 Loaded SERPAPI key from Colab secrets.")

except Exception:
    # ↓ fallback if not on Colab, or if you prefer manual entry
    os.environ["SERPAPI_API_KEY"] = getpass("SERPAPI API key (hidden): ")
    print("🔒 Loaded SERPAPI key from manual prompt.")


KeyboardInterrupt: Interrupted by user

In [7]:
# replace with the code above

#from google.colab import userdata
#import os
#os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

SecretNotFoundError: Secret SERPAPI_API_KEY does not exist.

In [26]:
agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [27]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search("Batman filming locations")                                                        
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'batman_locations = web_search("Batman filming locations")' due to: 
DuckDuckGoSearchException: https://html.duckduckgo.com/html 202 Ratelimit

[Step 1: Duration 4.16 seconds| Input tokens: 2,274 | Output tokens: 164]

...

[Step 3: Duration 6.55 seconds| Input tokens: 7,724 | Output tokens: 504]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Batman filming locations coordinates                                                                           
  batman_locations = {                                                                                             
      "Holborn tram tunnel, London": (51.5142, -0.1188),                                                           
      "Liverpool, UK": (53.4084, -2.9916),                                                                         
      "Chicago, USA": (41.8781, -87.6298),                                                                         
      "Temple Place, London": (51.5142, -0.1188),                                                                  
      "Watford, UK": (51.6596, -0.4045),                                                                           
      "Bedford, UK": (52.1326, -0.4797),                                                                           
      "Glasgow, UK": (55.8642, -4.2518),                                                                           
      "Shotts, UK": (55.8048, -3.8382)                                                                             
  }                                                                                                                
                                                                                                                   
  # Supercar factories coordinates                                                                                 
  supercar_factories = {                                                                                           
      "McLaren, Woking, UK": (51.3002, -0.6167),                                                                   
      "McLaren, Sheffield, UK": (53.3811, -1.4709),                                                                
      "McLaren, Bicester, UK": (51.6596, -0.4045),                                                                 
      "Ferrari, Maranello, Italy": (44.5646, 10.8846),                                                             
      "Porsche, Leipzig, Germany": (51.3397, 12.3731),                                                             
      "Pagani, San Cesario sul Panaro, Italy": (44.8333, 10.6833),                                                 
      "Maserati, Maranello, Italy": (44.5646, 10.8846),                                                            
      "Lamborghini, Sant'Agata Bolognese, Italy": (44.5208, 11.3389)                                               
  }                                                                                                                
                                                                                                                   
  # Gotham coordinates                                                                                             
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Function to calculate travel time for each location                                                            
  def calculate_travel_times(locations, origin_coords):                                                            
      travel_times = []                                                                                            
      for location, coords in locations.items():                                                                   
          travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=origin_coords)        
          travel_times.append((location, travel_time))                                                             
      [38;2;102

Execution logs:
Batman Filming Locations Travel Times:
                      Location  Travel Time (hours)
0  Holborn tram tunnel, London                 9.17
1                Liverpool, UK                 8.81
2                 Chicago, USA                 2.68
3         Temple Place, London                 9.17
4                  Watford, UK                 9.14
5                  Bedford, UK                 9.10
6                  Glasgow, UK                 8.60
7                   Shotts, UK                 8.64

Supercar Factories Travel Times:
                                   Location  Travel Time (hours)
0                       McLaren, Woking, UK                 9.13
1                    McLaren, Sheffield, UK                 8.95
2                     McLaren, Bicester, UK                 9.14
3                 Ferrari, Maranello, Italy                10.72
4                 Porsche, Leipzig, Germany                10.36
5     Pagani, San Cesario sul Panaro, Italy                10.68
6                Maserati, Maranello, Italy                10.72
7  Lamborghini, Sant'Agata Bolognese, Italy                10.77

Out - Final answer: (                      Location  Travel Time (hours)
0  Holborn tram tunnel, London                 9.17
1                Liverpool, UK                 8.81
2                 Chicago, USA                 2.68
3         Temple Place, London                 9.17
4                  Watford, UK                 9.14
5                  Bedford, UK                 9.10
6                  Glasgow, UK                 8.60
7                   Shotts, UK                 8.64,                                    Location  Travel Time 
(hours)
0                       McLaren, Woking, UK                 9.13
1                    McLaren, Sheffield, UK                 8.95
2                     McLaren, Bicester, UK                 9.14
3                 Ferrari, Maranello, Italy                10.72
4                 Porsche, Leipzig, Germany                10.36
5     Pagani, San Cesario sul Panaro, Italy                10.68
6                Maserati, Maranello, Italy                10.72
7  Lamborghini, Sant'Agata Bolognese, Italy                10.77)

[Step 4: Duration 11.20 seconds| Input tokens: 13,173 | Output tokens: 1,348]

In [ ]:
result

,Location,Travel Time to Gotham (hours)
0,"Necropolis Cemetery, Glasgow, Scotland, UK",8.60
1,"St. George's Hall, Liverpool, England, UK",8.81
2,"Two Temple Place, London, England, UK",9.17
3,"Wollaton Hall, Nottingham, England, UK",9.00
4,"Knebworth House, Knebworth, Hertfordshire, Eng...",9.15
5,"Acton Lane Power Station, Acton Lane, Acton, E...",9.16
6,"Queensboro Bridge, New York City, USA",1.01
7,"Wall Street, New York City, USA",1.00
8,"Mehrangarh Fort, Jodhpur, Rajasthan, India",18.34
9,"Turda Gorge, Turda, Romania",11.89


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [28]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- Gotham coordinates: 40.7128° N, 74.0060° W
- We need to find Batman filming locations worldwide.
- We need to calculate the travel time for a cargo plane to Gotham for each filming location.
- We need to find supercar factories with the same cargo plane transfer time to Gotham.

### 1.2. Facts to look up
- List of Batman filming locations worldwide.
  - Source: Web search for "Batman filming locations" and visit relevant websites.
- Coordinates of each Batman filming location.
  - Source: Visit the websites of the filming locations or use a mapping service like Google Maps.
- Coordinates of supercar factories worldwide.
  - Source: Web search for "supercar factories" and visit relevant websites.

### 1.3. Facts to derive
- Travel time for a cargo plane from each Batman filming location to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of Gotham and each filming 
location.
- Travel time for a cargo plane from each supercar factory to Gotham.
  - Derivation: Use the `calculate_cargo_travel_time` function with the coordinates of Gotham and each supercar 
factory.

## 2. Plan
1. Perform a web search for "Batman filming locations" to gather a list of filming locations.
2. For each Batman filming location, visit the source website to confirm and gather the coordinates.
3. Perform a web search for "supercar factories" to gather a list of supercar factories.
4. For each supercar factory, visit the source website to confirm and gather the coordinates.
5. Use the `calculate_cargo_travel_time` function to calculate the travel time from each Batman filming location to
Gotham.
6. Use the `calculate_cargo_travel_time` function to calculate the travel time from each supercar factory to 
Gotham.
7. Create a pandas dataframe containing the Batman filming locations, their coordinates, and the calculated travel 
time to Gotham.
8. Create a pandas dataframe containing the supercar factories, their coordinates, and the calculated travel time 
to Gotham.
9. Finalize the dataframes and prepare them for the final answer.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Perform a web search for "Batman filming locations"                                                    
  batman_filming_locations_search = web_search("Batman filming locations")                                         
  print(batman_filming_locations_search)                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of Gotham
City are located in Liverpool, although Chicago was ...

[Where The Batman Filmed (All Gotham Real-Life 
Locations)](https://screenrant.com/where-was-the-batman-movie-filmed/)
The Batman did its best to film in real-life locations, although prominent movie locations like the Batcave and 
Wayne Tower were built as sets on studio lots. When the movie was filming on location, the cast and crew mostly 
stayed in the UK to film the biggest Gotham locations.

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
Travel guide to film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
For The Batman, Matt Reeves mainly filmed in the UK and Chicago to capture the cold, gothic, and older feel of 
Gotham. While The Batman did try to film in real locations, iconic spots like the Batcave and Wayne Tower were 
built on studio sets. When filming on location, the cast and crew spent most of their time in the UK for the main 
Gotham scenes.

[Where Was The Batman Filmed? All the Iconic Locations 
Explored](https://otakukart.com/where-was-the-batman-filmed-all-the-iconic-locations-explored/)
The Batman, directed by Matt Reeves, is a gritty, noir-infused take on Gotham City, featuring Robert Pattinson as 
the latest incarnation of the Dark Knight. While the film's setting feels uniquely atmospheric, the shooting 
locations are scattered across the United Kingdom and Chicago, blending real-world settings with digital 
enhancements to bring Gotham to life. Though much of The Batman ...

[Batman Movie Locations You Can Visit In Person! - 
Travel](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Gotham city is fictional, but you can visit these batman movie locations to relive some of the iconic scenes.

[The Batman Filming Locations - filmiloc.com](https://www.filmiloc.com/the-batman-filming-locations/)
Matt Reeves' 'The Batman' was filmed in many places around the world to show a new version of Gotham City. This 
movie brings Robert Pattinson as Batman and shows his world. It also includes characters like Penguin, Catwoman, 
and others from DC Comics. Gotham City itself becomes a very important character in the movie. Filming on real 
locations helped create this new version of Gotham.

[Where Was The Batman Filmed? | iFILMthings](https://ifilmthings.com/where-was-the-batman-filmed/)
Where Was The Batman Filmed? The Batman movie was filmed in various locations around the world, including London 
and Liverpool in England, Chicago in the United States, and parts of Northern Ireland. In London, filming took 
place at iconic locations such as Trafalgar Square where a scene featuring Robert Pattinson's Bruce Wayne aboard 
his Batcycle was filmed, and around the city where ...

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
The Batman Filming Locations 'The Batman' was filmed in various parts of England and Scotland, specifically in 
Liverpool, London, Watford, Bedford, Glasgow, and Shotts.

[The Batman Filming Locations (2022)](https://a2zfilminglocation.com/the-batman-filming-locations-2022/)
The Batman Filming Locations: The Batman is a 2022 American superhe

...

────────────────────────────────────────────────── Updated plan ───────────────────────────────────────────────────
I still need to solve the task I was given:
```

You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in 
Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time.

```

Here are the facts I know and my new/updated plan of action to solve the task:
```
## 1. Updated facts survey
### 1.1. Facts given in the task
- Find all Batman filming locations in the world.
- Calculate the time to transfer via cargo plane to Gotham (40.7128° N, 74.0060° W).
- Return the data as a pandas dataframe.
- Also give some supercar factories with the same cargo plane transfer time.

### 1.2. Facts that we have learned
- Batman filming locations: Trafalgar Square, London; Liverpool, UK; Chicago, USA; Watford, UK; Bedford, UK; 
Glasgow, UK; Shotts, UK; Northern Ireland.
- Coordinates for Batman filming locations.
- Supercar factories: Porsche, Ferrari, Lamborghini, Pagani, McLaren, Rimac, Aston Martin, Koenigsegg, Bugatti, 
Hennessey.
- Coordinates for some supercar factories.

### 1.3. Facts still to look up
- Coordinates for remaining supercar factories (Hennessey).
- Confirm the coordinates for all Batman filming locations and supercar factories.
- Calculate the travel time for cargo planes to Gotham for each location.

### 1.4. Facts still to derive
- Filter supercar factories with the same cargo plane transfer time as Batman filming locations.

## 2. Plan
### 2.1. Confirm the coordinates for all Batman filming locations.
### 2.2. Confirm the coordinates for all supercar factories.
### 2.3. Calculate the travel time for cargo planes to Gotham for each Batman filming location.
### 2.4. Calculate the travel time for cargo planes to Gotham for each supercar factory.
### 2.5. Filter supercar factories with the same cargo plane transfer time as Batman filming locations.
### 2.6. Create a pandas dataframe with Batman filming locations and their travel times.
### 2.7. Create a pandas dataframe with supercar factories and their travel times.
### 2.8. Combine the dataframes and return the final answer.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
422 Client Error: Unprocessable Entity for url: https://router.huggingface.co/together/v1/chat/completions (Request
ID: nu33DsQ-4yUbBN-93fc36f3ceffdda4)

{'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 36446 `inputs` 
tokens and 2048 `max_new_tokens`', 'type': 'invalid_request_error', 'param': None, 'code': None}
{
  "id": "nu33DsQ-4yUbBN-93fc36f3ceffdda4",
  "error": {
    "message": "Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 36446 `inputs` 
tokens and 2048 `max_new_tokens`",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}

[Step 9: Duration 8.61 seconds| Input tokens: 132,625 | Output tokens: 6,783]

AgentGenerationError: Error in generating model output:
422 Client Error: Unprocessable Entity for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: nu33DsQ-4yUbBN-93fc36f3ceffdda4)

{'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 36446 `inputs` tokens and 2048 `max_new_tokens`', 'type': 'invalid_request_error', 'param': None, 'code': None}
{
  "id": "nu33DsQ-4yUbBN-93fc36f3ceffdda4",
  "error": {
    "message": "Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 36446 `inputs` tokens and 2048 `max_new_tokens`",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}


In [ ]:
detailed_report

,Location,Travel Time (hours)
0,"Bridge of Sighs, Glasgow Necropolis, Glasgow, ...",8.6
1,"Wishart Street, Glasgow, Scotland, UK",8.6


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [29]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        DuckDuckGoSearchTool(), #swapped Serpper for DuckDuckGo
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [31]:
import os
from getpass import getpass

try:
    # Colab path: pull from Tools → “User secrets…”
    from google.colab import userdata

    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    print("🔒 Loaded OPENAI API key from Colab secrets.")
except Exception:
    # Fallback: prompt you to paste it in (won’t echo to screen)
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key (hidden): ")
    print("🔒 Loaded OPENAI API key from manual prompt.")


OpenAI API key (hidden): ··········
🔒 Loaded OPENAI API key from manual prompt.


In [30]:
#replace with a way to add secret keys above instead of hardcoding in the api key
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

SecretNotFoundError: Secret OPENAI_API_KEY does not exist.

In [32]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [33]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
<think>
Okay, let's tackle this task step by step. The user wants Batman filming locations, their cargo plane travel times 
from Gotham (40.7128° N, 74.0060° W), and supercar factories with similar travel times. Then, they need a map 
visualizing these points with color-coded travel times.

First, I need to figure out the facts required. The given facts are the coordinates of Gotham, which is actually 
New York City's coordinates. The task mentions using a cargo plane's cruising speed, which the tool defaults to 750
km/h if not specified. 

For the facts to look up, I need a list of all Batman filming locations. Since Batman movies have been filmed in 
various places over the years, I'll need to use the web_agent to search for each movie's filming locations. For 
example, "The Dark Knight" had scenes in Chicago, London, etc. Each movie's locations need to be compiled. Then, 
each location's coordinates (latitude and longitude) must be obtained. Maybe using geocoding via the web_agent or a
service like Google Maps.

Next, supercar factories. These are companies like Ferrari (Italy), Lamborghini (Italy), Bugatti (France), McLaren 
(UK), Pagani (Italy), Koenigsegg (Sweden), etc. But I need to confirm their factory locations and get their 
coordinates. Again, the web_agent can help here.

Once all locations are gathered, the next step is to calculate the travel time from Gotham to each using the 
calculate_cargo_travel_time function. This function uses great-circle distance and the cruising speed. The result 
is the time in hours.

After calculating times for Batman locations, I need to find supercar factories with similar times. The user wants 
at least 6 points in total, so maybe 3 Batman locations and 3 factories, but more if needed. The key is to match 
the cargo times. For example, if a Batman location takes 7 hours, find factories that also take around 7 hours.

Then, plotting all these points on a map. Using plotly's scatter_map, each point's color will represent the travel 
time. The example provided uses a DataFrame with lat, lon, name, and color based on 'peak_hour' (which we'll 
replace with travel time). The map needs to be saved as saved_map.png.

Potential issues: Some filming locations might be in the same city, leading to duplicate coordinates. Need to 
ensure each entry is unique. Also, ensuring that the web_agent fetches accurate coordinates. If a location's 
coordinates are incorrect, the travel time calculation will be off. For supercar factories, some might be in the 
same country, so their times might cluster. Need to check enough factories to get 6 points.

The plan would be:

1. Use web_agent to find all Batman movie filming locations. Compile a list.
2. For each location, get coordinates via web_agent.
3. Calculate travel time from Gotham to each Batman location.
4. Use web_agent to find supercar factory locations and their coordinates.
5. Calculate travel times for these factories.
6. Compare factory times with Batman locations' times to find matches. Select at least 6 points in total.
7. Combine both sets of locations into a DataFrame with lat, lon, name, and travel time.
8. Plot using plotly, color by travel time, save as saved_map.png.
9. Return the figure using final_answer.

Need to make sure that the web_agent is tasked with specific queries. For example, "List all filming locations for 
Batman movies" and "List manufacturing locations of supercar companies with their coordinates." Also, handle any 
discrepancies in data, like if a factory has multiple locations, pick the main one. The example code uses a 
DataFrame, so structuring the data correctly is crucial. The color scale should be based on the travel time, so the
'color' parameter in px.scatter_map should be set to the travel time column.


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
<think>                                                                                                            
Okay, let's tackle this step by step. First, I need to get all the Batman filming locations. I'll ask the web_agent
to find them along with their coordinates. Then, I'll calculate the travel time from Gotham (NYC) to each location 
using the calculate_cargo_travel_time tool.                                                                        
                                                                                                                   
Next, I need supercar factories. I'll have the web_agent list those and their coordinates too. Again, calculate    
their travel times from Gotham. Then, I have to match factories to Batman locations where their travel times are   
similar. The goal is at least six points total, so if there are three Batman locations, I need three factories with
matching times, or adjust accordingly.                                                                             
                                                                                                                   
Once I have all the data, I'll compile it into a DataFrame. Using Plotly, I'll create a scatter map with colors    
based on travel time. Finally, save the map and return it as the answer. Let me make sure each step is done        
correctly, handling possible errors like missing coordinates or no matching factories.                             
</think>                                                                                                           
                                                                                                                   
Thought: First, I need to get all Batman filming locations with coordinates. I'll use the web_agent to find this   
information.                                                                                                       
Code:                                                                                                              
```py                                                                                                              
batman_task 

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_task = "Find all real-world filming locations for Batman movies (1989-2023) with exact                    
  latitude/longitude coordinates. Include both cities and specific landmarks."                                     
  batman_locations = web_agent(batman_task)                                                                        
  print("Batman filming locations:", batman_locations)                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'for movie in batman_movies:
    search_query = f"filming locations {movie}"
    search_results = web_search(search_query)
    locations[movie\] = search_results' due to: DuckDuckGoSearchException: https://html.duckduckgo.com/html 202 
Ratelimit

[Step 1: Duration 61.72 seconds| Input tokens: 3,223 | Output tokens: 311]

KeyboardInterrupt: 

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)